# Operational Research Assignment 2025
## Planning of weekly course timetable
### Lolos Ioannis - 10674
### lolosioann@ece.auth.gr

In [49]:
from gurobipy import Model, GRB
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches

### Data Representation for the problem

The cell below defines the core data structures used for the scheduling problem. It includes:

- A list of teachers.
- A dictionary of subjects-teachers key-value pairs, including special cases like multiple Math and PE teachers.
- Two classes (`C1` and `C2`) and the weekly schedule defined by days and time slots (5 days × 4 slots/day).
- A specification of required teaching hours for each subject-class pair.
- Special cases like teacher specific unavailability periods and reserved time slots for PE and weekly planning.

These data will be used in our optimization model.

In [50]:
# Data representation
teachers = [
    "Gesmanidis",       
    "Insulina",   
    "Chartoula",  
    "Lathopraxis",
    "Antiparagogos",
    "Kirkofidou",
    "Platiazon",
    "Bratsakis",
    "Trehalitoula"
]

# we use math_1 and math_2 to represent the two different math teachers
# and the same for PE_1 and PE_2
subjects = {
    "English": "Gesmanidis",
    "Biology": "Insulina",
    "History": "Chartoula",
    "Math_1": "Lathopraxis",      
    "Math_2": "Antiparagogos",    
    "Physics": "Kirkofidou",
    "Philosophy": "Platiazon",
    "PE_1": "Bratsakis",          
    "PE_2": "Trehalitoula"        
}

# two classes (tmhmata): C1 and C2
classes = ["C1", "C2"]

days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
slots_per_day = ["08_10", "10_15_12_15", "14_16", "16_15_18_15"]

# 5x4 grid
time_slots = [(d, s) for d in days for s in slots_per_day]

# Format -> subject: (C1_hours, C2_hours)
teaching_hours = {     
    "English": (1, 1),
    "Biology": (3, 3),
    "History": (2, 2),
    "Math_1": (0, 4),   
    "Math_2": (4, 0),   
    "Physics": (3, 3),
    "Philosophy": (1, 1),
    "PE_1": (1, 0),
    "PE_2": (0, 1)
}

# Fixed slot reserved for study planning
reserved_slots = [("Mon", "08_10")]

# PE must be on Thursday 14:00-16:00
pe_slot = ("Thu", "14_16")

# Teacher unavailability
unavailable = {
    "Lathopraxis": [("Mon", "08_10"), ("Mon", "10_15_12_15")],
    "Insulina": [("Wed", s) for s in slots_per_day]
}

### Model Initialization and Decision Variables

Next, we initialize a Gurobi optimization model for problem. The decision variables `x[class, subject, time_slot]` are binary. Their value is 1 if a given subject for a specific section is scheduled at a particular time slot, else 0.

Key components:
- **Model**: A Gurobi `Model` named `"timetable"` is created.
- **Variables**: A binary variable is defined for every combination of class, subject, and time slot.
- **Objective**: A dummy objective (`0`) is set since the problem requires us to only find a feasible schedule, with no cost to optimize.


In [51]:
# Create the model
x = {}  # Decision variables: x[class, subject, time_slot]
model = Model("timetable")

for c in classes:
    for sub in teaching_hours:
        for t in time_slots:
            var_name = f"x_{c}_{sub}_{t[0]}_{t[1]}"
            x[c, sub, t] = model.addVar(vtype=GRB.BINARY, name=var_name)

# there is nothing to minimize, but we need to set an objective function
model.setObjective(0, GRB.MINIMIZE)

### Constraints

1. **Subject Teaching Hours**: Each subject must be scheduled for a fixed number of hours (`H`) per class (`C1`, `C2`), as specified in `teaching_hours`.  
   Since a scheduled subject for a time slot is encoded as `x[class, subject, timeslot] == 1`, the sum over all time slots must equal the required number of hours:  
   `sum(x[c, sub, t] for t in time_slots) == H`.

2. **Class Time Slot Exclusivity**: A class can attend at most one subject per time slot.  
   For each class and time slot, the sum over all subjects must be at most 1:  
   `sum(x[c, sub, t] for sub in teaching_hours) <= 1`.

3. **Teacher Availability**: A teacher cannot be scheduled to teach more than one class at the same time.  
   For each teacher and time slot, the sum over all sections and their subjects taught by that teacher must be at most 1:  
   `sum(x[c, sub, t] for c in classes for sub if subjects[sub] == teacher) <= 1`.

4. **Subject Frequency per Day**: A subject can be scheduled at most once per day for each class.  
   For each class, subject, and day, the sum over time slots on that day must be at most 1:  
   `sum(x[c, sub, t] for t in time_slots if t[0] == day) <= 1`.

5. **Reserved Time Slot**: Reserved slots (e.g., Monday 08:00–10:00) are excluded from teaching for all subjects and classes.  
   For each reserved slot, the corresponding variable must be 0:  
   `x[c, sub, t] == 0 for all c, sub, and t in reserved_slots`.

6. **Physical Education Timing**: PE classes must take place on Thursday from 14:00–16:00.  
   All variables for PE outside that specific slot must be 0:  
   For all `t ≠ ("Thu", "14_16")`,  
   `x["C1", "PE_1", t] == 0` and `x["C2", "PE_2", t] == 0`.

7. **Teacher Unavailability**: Teachers cannot be scheduled during their unavailable time slots.  
   For each teacher, class, and subject they teach, the corresponding variables in their unavailable slots must be 0:  
   `x[c, sub, t] == 0 for t in unavailable[teacher] and subjects[sub] == teacher`.


In [52]:
# Constraints
# Each subject must be scheduled exactly H times for each class c:
for sub, (h1, h2) in teaching_hours.items():
    for c_idx, c in enumerate(classes):
        H = h1 if c_idx == 0 else h2
        model.addConstr(sum(x[c, sub, t] for t in time_slots) == H)


# Each class can attend only one subject at a time:
for s in classes:
    for t in time_slots:
        model.addConstr(sum(x[c, sub, t] for sub in teaching_hours) <= 1)


# A teacher must not be scheduled to teach two classes at the same time.
for t in time_slots:
    for teacher in teachers:
        teaching_subs = [sub for sub, tchr in subjects.items() if tchr == teacher]
        model.addConstr(
            sum(x[c, sub, t] for c in classes for sub in teaching_subs) <= 1
        )

# No more than one lesson per subject per section per Day
for c in classes:
    for sub in teaching_hours:
        for day in days:
            slots_today = [t for t in time_slots if t[0] == day]
            model.addConstr(sum(x[c, sub, t] for t in slots_today) <= 1)

# fixed slot reserved for study planning
for s in classes:
    for sub in teaching_hours:
        for t in reserved_slots:
            model.addConstr(x[c, sub, t] == 0)

# PE must be on Thursday 14:00-16:00
for t in time_slots:
    if t != pe_slot:
        model.addConstr(x["C1", "PE_1", t] == 0)
        model.addConstr(x["C2", "PE_2", t] == 0)

# Teachers' unavailability
for teacher, unavailable_slots in unavailable.items():
    teaching_subs = [sub for sub, tchr in subjects.items() if tchr == teacher]
    for c in classes:
        for sub in teaching_subs:
            for t in unavailable_slots:
                model.addConstr(x[c, sub, t] == 0)


### Solve the Model

This cell runs the optimization process using Gurobi and checks the status of the solution:

- `model.setParam('OutputFlag', 0)`: Suppresses the solver's detailed output for a cleaner notebook interface.
- `model.optimize()`: Starts the solver to find a feasible timetable that satisfies all defined constraints.
- `model.status`: After solving, this attribute holds the status of the solution.

If the model status is `GRB.OPTIMAL`, an feasible schedule has been found that satisfies all constraints. Otherwise, the code reports that no solution exists.


In [53]:
model.setParam('OutputFlag', 0) # Suppress output
model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution found!\n")
else:
    print("No solution found.")


Optimal solution found!



### Timetable Construction and Display

The bellow cell constructs and prints the timetable for each class (`C1` and `C2`) based on the optimized decision variables.
For each class (`C1`, `C2`), the function builds the timetable and prints it in tabular form.

This provides a readable weekly schedule for each class based on the optimized solution.

In [ ]:
def build_timetable_for_class(cls):
    """
    Creates a `pandas.DataFrame` with time slots as rows and days as columns.
    For each time slot, it checks which subject is scheduled (i.e., the corresponding 
    binary variable `x[cls, sub, time_slot]` is set to 1) and fills 
    in the subject name. If no subject is assigned, a placeholder `"---"` is used.
    """
    table = pd.DataFrame(index=slots_per_day, columns=days)

    for day in days:
        for slot in slots_per_day:
            t = (day, slot)
            subject = "---"
            for sub in teaching_hours:
                if x[cls, sub, t].X > 0.5:
                    subject = sub
                    break
            table.at[slot, day] = subject

    return table

for cls in classes:
    print(f"\nTimetable for {cls}")
    timetable = build_timetable_for_class(cls)
    print(timetable)



Timetable for C1
                 Mon      Tue         Wed      Thu      Fri
08_10         Math_2      ---     Physics  Biology   Math_2
10_15_12_15      ---  Physics  Philosophy  History      ---
14_16            ---      ---      Math_2     PE_1  English
16_15_18_15  Biology      ---         ---      ---  Biology

Timetable for C2
                 Mon      Tue     Wed      Thu         Fri
08_10            ---  History  Math_1  English     Biology
10_15_12_15  Physics   Math_1     ---      ---  Philosophy
14_16        Biology  Biology     ---     PE_2     Physics
16_15_18_15   Math_1  Physics     ---   Math_1     History
